<a href="https://colab.research.google.com/github/sebapusch/rnns/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset

dataset = load_dataset("stanfordnlp/sst2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [24]:
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [7]:
train_dataset[0]

{'idx': 0,
 'sentence': 'hide new secretions from the parental units ',
 'label': 0}

In [8]:
for i in range(5):
  print(f"{i}: {train_dataset[i]}")

0: {'idx': 0, 'sentence': 'hide new secretions from the parental units ', 'label': 0}
1: {'idx': 1, 'sentence': 'contains no wit , only labored gags ', 'label': 0}
2: {'idx': 2, 'sentence': 'that loves its characters and communicates something rather beautiful about human nature ', 'label': 1}
3: {'idx': 3, 'sentence': 'remains utterly satisfied to remain the same throughout ', 'label': 0}
4: {'idx': 4, 'sentence': 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ', 'label': 0}


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
def tokenizer_text(examples):
  return tokenizer(
      examples["sentence"],
      truncation=True,
      padding=True,
      max_length=128,  # adjustable
      return_tensors=None  # Returns Python lists
  )

In [11]:
tokenized_train = train_dataset.map(tokenizer_text, batched=True)
tokenized_val = val_dataset.map(tokenizer_text, batched=True)
tokenized_test = test_dataset.map(tokenizer_text, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [13]:
print(tokenized_train[0])

{'idx': 0, 'sentence': 'hide new secretions from the parental units ', 'label': 0, 'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [14]:
tokenized_train.features

{'idx': Value('int32'),
 'sentence': Value('string'),
 'label': ClassLabel(names=['negative', 'positive']),
 'input_ids': List(Value('int32')),
 'token_type_ids': List(Value('int8')),
 'attention_mask': List(Value('int8'))}

In [15]:
columns_to_remove = ['idx', 'sentence', 'token_type_ids']

tokenized_train = tokenized_train.remove_columns(columns_to_remove)
tokenized_val = tokenized_val.remove_columns(columns_to_remove)
tokenized_test = tokenized_test.remove_columns(columns_to_remove)

In [16]:
tokenized_train.features

{'label': ClassLabel(names=['negative', 'positive']),
 'input_ids': List(Value('int32')),
 'attention_mask': List(Value('int8'))}

In [18]:
print(tokenized_train[0])

{'label': 0, 'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [30]:
def prepare_datasets(prcteprocess_funion):
  dataset = load_dataset("stanfordnlp/sst2")

  train_dataset = dataset["train"]
  val_dataset = dataset["validation"]
  test_dataset = dataset["test"]

  tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

  tokenized_train = train_dataset.map(preprocess_function, batched=True)
  tokenized_val = val_dataset.map(preprocess_function, batched=True)
  tokenized_test = test_dataset.map(preprocess_function, batched=True)

  columns_to_remove = ['idx', 'sentence', 'token_type_ids']

  tokenized_train = tokenized_train.remove_columns(columns_to_remove)
  tokenized_val = tokenized_val.remove_columns(columns_to_remove)
  tokenized_test = tokenized_test.remove_columns(columns_to_remove)

  return tokenized_train, tokenized_val, tokenized_test


In [27]:
tokenized_train, tokenized_val, tokenized_test = prepare_datasets(tokenizer_text)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

In [33]:
print(tokenized_val[0])
print(tokenized_val.features)

{'label': 1, 'input_ids': [101, 2009, 1005, 1055, 1037, 11951, 1998, 2411, 12473, 4990, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'label': ClassLabel(names=['negative', 'positive']), 'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8'))}


In [20]:
# from transformers import DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)